In [1]:
import import_ipynb
import numpy as np
from numpy import genfromtxt

In [57]:
def save_partitions_to_file(kdnodes, path):
    '''
    stretch sub_domains in to one row
    @kdnodes[i][sub_domains, len(sub_dataset1), id, parent_id, left_child_id, right_child_id]
    return:
    @stretched_kdnodes [num_dims, l1,l2,...,ln, u1,u2,...,un, size, id, pid, left_child,id, right_child_id]
    '''
    stretched_kdnodes = []
    num_dims = len(kdnodes[0][0])
    
    for i in range(len(kdnodes)):
        row = [num_dims]
        domains_lower = [d[0] for d in kdnodes[i][0]]
        domains_upper = [d[1] for d in kdnodes[i][0]]
        
        for dl in domains_lower:
            row.append(dl)
        for du in domains_upper:
            row.append(du)
        
        row.append(kdnodes[i][1])
        
        # to be compatible with qd-tree's partition, that do not have the last 4 attributes
        if len(kdnodes[i]) > 2:
            row.append(kdnodes[i][-4])
            row.append(kdnodes[i][-3])
            row.append(kdnodes[i][-2])
            row.append(kdnodes[i][-1])
            
        stretched_kdnodes.append(row)
    
    stretched_kdnodes = np.array(stretched_kdnodes)
    np.savetxt(path, stretched_kdnodes, delimiter=',')
    return stretched_kdnodes

def load_partitions_from_file(path):
    '''
    the loaded stretched_kdnodes: [num_dims, l1,l2,...,ln, u1,u2,...,un, size, id, pid, left_child,id, right_child_id]
    '''
    stretched_kdnodes = genfromtxt(path, delimiter=',')
    num_dims = int(stretched_kdnodes[0,0])
    kdnodes = []
    
    for i in range(len(stretched_kdnodes)):
        domains = [ [stretched_kdnodes[i,k+1],stretched_kdnodes[i,2*k+1]] for k in range(num_dims) ]
        row = [domains]
        row.append(stretched_kdnodes[i,2*num_dims+1])
        
        # to be compatible with qd-tree's partition, that do not have the last 4 attributes
        if len(stretched_kdnodes[i]) > 2*num_dims+2:
            row.append(stretched_kdnodes[i,-4])
            row.append(stretched_kdnodes[i,-3])
            row.append(stretched_kdnodes[i,-2])
            row.append(stretched_kdnodes[i,-1])
    
        kdnodes.append(row)
    
    return kdnodes

In [64]:
def save_partition_dict_to_file(kdnode_dict, path):
    stretched_kdnodes = []
    
    for pid, partition in kdnode_dict.items():
        row = [len(partition[0])]
        domains_lower = [d[0] for d in partition[0]]
        domains_upper = [d[1] for d in partition[0]]

        for dl in domains_lower:
            row.append(dl)
        for du in domains_upper:
            row.append(du)
            
        row.append(partition[-5])
        row.append(partition[-4])
        row.append(partition[-3])
        row.append(partition[-2])
        row.append(partition[-1])
            
        stretched_kdnodes.append(row)
        
    stretched_kdnodes = np.array(stretched_kdnodes)
    np.savetxt(path, stretched_kdnodes, delimiter=',')
    return stretched_kdnodes

def load_partition_dict_from_file(path):
    '''
    return:
    @kdnode_dict: pid: kdnode
    @kdnodes: all the kdnode is the above dict, inlcuding those non-leaf records
    '''
    stretched_kdnodes = genfromtxt(path, delimiter=',')
    num_dims = int(stretched_kdnodes[0,0])
    kdnodes = [] # notice the kdnodes here may contain those non-leaf records!!!
    kdnode_dict = {}
    
    for i in range(len(stretched_kdnodes)):
        domains = [ [stretched_kdnodes[i,k+1],stretched_kdnodes[i,2*k+1]] for k in range(num_dims) ]
        row = [domains]
        row.append(stretched_kdnodes[i,-5])
        row.append(stretched_kdnodes[i,-4])
        row.append(stretched_kdnodes[i,-3])
        row.append(stretched_kdnodes[i,-2])
        row.append(stretched_kdnodes[i,-1])
    
        kdnodes.append(row)
        kdnode_dict.update({row[-4]:row})
    
    return kdnode_dict, kdnodes

In [58]:
# test_partition = [ [[[0,1],[0,2]],10,1,-1,2,3],
#                  [[[0,11],[0,22]],10,2,-1,4,5],
#                  [[[0,121],[0,232]],10,3,-1,7,8]]
# sp = save_partitions_to_file(test_partition, 'C:/Users/Cloud/iCloudDrive/HUAWEI_LKD/Dataset/Legacy/data/test.csv')
# print(sp)
# kdnodes = load_partitions_from_file('C:/Users/Cloud/iCloudDrive/HUAWEI_LKD/Dataset/Legacy/data/test.csv')
# print(kdnodes)

In [61]:
# kdnode_dict = {1: [[[0,1],[0,2]],10,1,-1,2,3],
#               2: [[[0,11],[0,22]],10,2,-1,4,5],
#               3:[[[0,121],[0,232]],10,3,-1,7,8]}
# kdnodes = save_partition_dict_to_file(kdnode_dict, 'C:/Users/Cloud/iCloudDrive/HUAWEI_LKD/Dataset/Legacy/data/test.csv')
# print(kdnodes)
# kdn_dict, kdn = load_partition_dict_from_file('C:/Users/Cloud/iCloudDrive/HUAWEI_LKD/Dataset/Legacy/data/test.csv')
# print(kdn_dict)
# print(kdn)